Načtení dat

In [2]:
import pandas as pd
data = pd.read_csv("diabetes_prediction_dataset.csv")
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


# Rozdělení na training a testing set

In [4]:
from sklearn.model_selection import train_test_split
X = data.drop('diabetes', axis=1)
y = data['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set: ", X_train.shape)
print("Testing set: ", X_test.shape)

Training set:  (80000, 8)
Testing set:  (20000, 8)


# Doplnění chybějících pozorování pro smoking_history

In [ ]:
print(X_train['smoking_history'].value_counts())
print("celkem: " ,X_train['smoking_history'].shape[0])

NameError: name 'X_train' is not defined

In [ ]:
import numpy as np
value_counts = X_train[X_train['smoking_history'] != 'No Info']['smoking_history'].value_counts()
total_count = value_counts.sum()
distribution = (X_train['smoking_history'].value_counts() / total_count).drop('No Info')
random= np.random.choice(distribution.index,X_train[X_train['smoking_history'] == 'No Info'].shape[0] ,p=distribution.values)

X_train['smoking_history'].loc[X_train['smoking_history'] == 'No Info'] = random


C:\Users\filip\AppData\Local\Temp\ipykernel_30960\920219924.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  X_train['smoking_history'].loc[X_train['smoking_history'] == 'No Info'] = random
C:\Users\filip\AppData\Local\Temp\ipykernel_3096

In [ ]:
print(X_train['smoking_history'].value_counts())
print("celkem: " ,X_train['smoking_history'].shape[0])

smoking_history
never          43780
former         11692
current        11532
not current     7978
ever            5018
Name: count, dtype: int64
celkem:  80000


In [ ]:
#Jednodušší doplnění módusem
#X_train['smoking_history'] = X_train['smoking_history'].replace('No Info', str(X_train['smoking_history'].mode()[0]))
#X_train['smoking_history']

# Upsampling

In [ ]:
#upsample the testing data to contain more samples with diabetes = 1
from sklearn.utils import resample
X_train['diabetes'] = y_train
#Separate the features and target variable
X,y= X_train.drop('diabetes', axis=1), X_train['diabetes']
#Resampling the minority class (diabetes = 1) by up-sampling
X_up, y_up = resample(X[y==1], y[y==1], replace=True, n_samples=X[y==0].shape[0], random_state=42)
#Combining the upsampled data with the original data of the majority class
X_resampled = pd.concat([X[y==0], X_up])
y_resampled = pd.Series(np.concatenate([y[y==0], y_up]))
print("Počet kde diabetes = 1 po upsamplingu: ", y_resampled.sum()) 

X_train = X_resampled
y_train = y_resampled

Počet kde diabetes = 1 po upsamplingu:  73208


# Odstranění dat

Odstranění záznamů, kde je pohlaví určeno jako Other. Dataset obsahuje těchto instancí jen 16.

In [ ]:
print ("Počet pro pohláví 'Other'",X_train[X_train['gender'] == 'Other'].shape[0])
X_train = X_train.drop(X_train[X_train['gender'] == 'Other'].index)

Počet pro pohláví 'Other' 16


# Převedení kategoriálních hodnot na binární

## Gender na binarní reprezentaci

In [7]:
X_train = pd.get_dummies(X_train, columns=['gender'])
X_test = pd.get_dummies(X_test, columns=['gender'])
X_test

,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,gender_Other
75721,13.0,0,0,No Info,20.82,5.8,126,True,False,False
80184,3.0,0,0,No Info,21.00,5.0,145,True,False,False
19864,63.0,0,0,former,25.32,3.5,200,False,True,False
76699,2.0,0,0,never,17.43,6.1,126,True,False,False
92991,33.0,0,0,not current,40.08,6.2,200,True,False,False
...,...,...,...,...,...,...,...,...,...,...
32595,44.0,0,0,not current,21.95,6.0,159,True,False,False
29313,61.0,1,0,former,41.98,3.5,90,False,True,False
37862,49.0,0,0,never,26.51,4.8,100,True,False,False
53421,73.0,0,1,No Info,27.32,6.6,100,False,True,False


## Smoking_history na binární reprezentaci

In [8]:
X_train = pd.get_dummies(X_train, columns=['smoking_history'])
X_test = pd.get_dummies(X_test, columns=['smoking_history'])
X_test

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,gender_Other,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
75721,13.0,0,0,20.82,5.8,126,True,False,False,True,False,False,False,False,False
80184,3.0,0,0,21.00,5.0,145,True,False,False,True,False,False,False,False,False
19864,63.0,0,0,25.32,3.5,200,False,True,False,False,False,False,True,False,False
76699,2.0,0,0,17.43,6.1,126,True,False,False,False,False,False,False,True,False
92991,33.0,0,0,40.08,6.2,200,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32595,44.0,0,0,21.95,6.0,159,True,False,False,False,False,False,False,False,True
29313,61.0,1,0,41.98,3.5,90,False,True,False,False,False,False,True,False,False
37862,49.0,0,0,26.51,4.8,100,True,False,False,False,False,False,False,True,False
53421,73.0,0,1,27.32,6.6,100,False,True,False,True,False,False,False,False,False


# Clustering

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
kmeans = KMeans(n_clusters=5, random_state=42,n_init=10)
kmeans.fit(X)
labels = kmeans.labels_